<a href="https://colab.research.google.com/github/56sarager/Colabs/blob/main/Implied_Volatility_Section_III_Matplotlib_Plots_and_Animations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brent Method vs. Newton-Raphson Method (Plots Stationary, Includes Animation)
The first cell below takes as input a single ticker symbol and generates two non-interactive Matplotlib plots of the implied volatility calculated using the Brent Method and the Newton-Raphson Method. It is noted that the methods yield different results, possibly by design of the algorithms, due to numerical artifacts, or for some other reason. This is to be looked into later. The second cell below creates a gif of the previously stationary Brent Method plot using historical data from Yahoo Finance in 15 minute increments. It takes about 5 minutes to run. The thrid cell below does the same as the second but for the Newton-Raphson Method and takes about 15 minutes to run. The gifs are not displayed but saved to the files tab of Colab. For those who are pressed for time, sample gifs are saved to the GitHub where this Colab can be found.

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from scipy.interpolate import griddata

# Black-Scholes formula for option pricing
def black_scholes_call(S, K, T, r, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(option_price, S, K, T, r):
    # Define the objective function for Brent's method
    def objective(sigma):
        return black_scholes_call(S, K, T, r, sigma) - option_price
    try:
        # Use Brent's method to find the implied volatility
        return brentq(objective, 1e-6, 4)
    except ValueError:
        return np.nan

# Vega function to compute the derivative of the option price with respect to volatility
def vega(S, K, T, r, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Function to calculate implied volatility using the Newton-Raphson method
def implied_volatility_newton(option_price, S, K, T, r, option_type='call', max_iter=100, tol=1e-6):
    # Initial guess for volatility (usually in the range of 20%)
    sigma = 0.2

    for i in range(max_iter):
        if option_type == 'call':
            price = black_scholes_call(S, K, T, r, sigma)

        # Calculate the Vega (the derivative of the option price with respect to volatility)
        vega_value = vega(S, K, T, r, sigma)

        # Newton-Raphson update rule
        diff = price - option_price
        if abs(diff) < tol:
            return sigma  # Return the implied volatility if the difference is small enough

        sigma -= diff / vega_value  # Update sigma using the Newton-Raphson formula

    return np.nan  # Return NaN if the method fails to converge within the max iterations

surface_data2 = []

# Step 1: Download data
ticker = "SPY"
ticker = input(f"Enter ticker symbol (default: {ticker}): ") or ticker
stock = yf.Ticker(ticker)
options_dates = stock.options

# Step 2: Get current stock price
current_price = stock.history(period="1d")['Close'].iloc[-1]

# Set up parameters
risk_free_rate = 0.0425
surface_data = []

# Step 3: Loop through option expiration dates and strikes
for exp_date in options_dates[:5]:  # Limit to 5 expiration dates for simplicity
    options_chain = stock.option_chain(exp_date)
    calls = options_chain.calls
    expiration = (datetime.strptime(exp_date, "%Y-%m-%d") - datetime.now()).days / 365.0

    for index, row in calls.iterrows():
        strike = row['strike']
        market_price = row['lastPrice']

        # Calculate implied volatility
        iv = implied_volatility(market_price, current_price, strike, expiration, risk_free_rate)
        iv_call = implied_volatility_newton(market_price, current_price, strike, expiration, risk_free_rate, option_type='call')
        surface_data2.append((expiration, strike, iv_call))

        # Append data for surface plot
        surface_data.append((expiration, strike, iv))

    for index, row in calls.iterrows():
        strike = row['strike']
        market_price = row['lastPrice']

# Convert surface data to DataFrame
df_surface = pd.DataFrame(surface_data, columns=['Expiration', 'Strike', 'ImpliedVolatility']).dropna()

# Step 4: Create a 3D surface plot
fig = plt.figure(figsize=(18, 7))

# Create first subplot for Brent's method
ax1 = fig.add_subplot(121, projection='3d')

# Create a meshgrid for the surface plot
x = df_surface['Strike']
y = df_surface['Expiration']
z = df_surface['ImpliedVolatility']

# Define grid for smoother surface
x_grid = np.linspace(x.min(), x.max(), 100)
y_grid = np.linspace(y.min(), y.max(), 100)
X, Y = np.meshgrid(x_grid, y_grid)

# Interpolate the implied volatilities
Z = griddata((x, y), z, (X, Y), method='cubic')

# Plot the surface
surf = ax1.plot_surface(X, Y, Z/100, cmap='copper', edgecolor='none')

ax1.set_xlabel('Strike Price (USD)')
ax1.set_ylabel('Time to Expiry (years)')
ax1.set_zlabel('Implied Volatility')
ax1.set_title(f'Implied Volatility Surface for {ticker} (Brent)')
fig.colorbar(surf, shrink=0.5, aspect=5, ax=ax1)

# Convert surface data to DataFrame
df_surface2 = pd.DataFrame(surface_data2, columns=['Expiration', 'Strike', 'ImpliedVolatility']).dropna()

# Prepare data for Matplotlib 3D surface plot
x2 = df_surface2['Strike']
y2 = df_surface2['Expiration']
z2 = df_surface2['ImpliedVolatility']

# Define grid for smoother surface
x_grid2 = np.linspace(x2.min(), x2.max(), 100)
y_grid2 = np.linspace(y2.min(), y2.max(), 100)
X2, Y2 = np.meshgrid(x_grid2, y_grid2)

# Interpolate the implied volatilities
Z2 = griddata((x2, y2), z2, (X2, Y2), method='cubic')

# Create second subplot for Newton-Raphson method
ax2 = fig.add_subplot(122, projection='3d')

# Plot the surface
surf2 = ax2.plot_surface(X2, Y2, Z2, cmap='copper', edgecolor='none')

# Customize plot labels and title
ax2.set_xlabel('Strike Price (USD)')
ax2.set_ylabel('Time to Expiry (years)')
ax2.set_zlabel('Implied Volatility')
ax2.set_title(f'Implied Volatility Surface for {ticker} (Newton-Raphson)')

# Add a color bar
fig.colorbar(surf2, shrink=0.5, aspect=5, ax=ax2)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Takes slightly over 5 minutes to run
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import brentq
from datetime import datetime, timedelta
from scipy.interpolate import griddata
from PIL import Image

# Black-Scholes formula for option pricing
def black_scholes_call(S, K, T, r, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(option_price, S, K, T, r):
    def objective(sigma):
        return black_scholes_call(S, K, T, r, sigma) - option_price
    try:
        return brentq(objective, 1e-6, 4)
    except ValueError:
        return np.nan

# Parameters
ticker = "SPY"
risk_free_rate = 0.0425
frames_per_second = 4
interval = '15m'
history_duration = '5d'

# Download historical 15-minute interval data
stock = yf.Ticker(ticker)
historical_data = stock.history(interval=interval, period=history_duration)

# Get options data (using expiration dates)
options_dates = stock.options

# Prepare figure for each frame and save as image
fig = plt.figure(figsize=(14, 6))
ax = fig.add_subplot(111, projection='3d')
cbar_ax = fig.add_axes([0.9, 0.15, 0.02, 0.7])  # Position for color bar
images = []  # List to store frames for GIF

# Create a ScalarMappable for the color bar
norm = plt.Normalize(vmin=0, vmax=1)  # Initialize with dummy values
cmap = cm.bone
mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(mappable, cax=cbar_ax, label="Implied Volatility")

# Loop through each 15-minute interval to prepare data for each frame
for i in range(len(historical_data)):
    interval_time = historical_data.index[i]
    current_price = historical_data['Close'].iloc[i]

    surface_data = []
    for exp_date in options_dates[:5]:  # Limit to 5 expiration dates for simplicity
        expiration_date = pd.Timestamp(exp_date).tz_localize('America/New_York')
        expiration = (expiration_date - interval_time).days / 365.0  # Days to expiration in years

        options_chain = stock.option_chain(exp_date)
        calls = options_chain.calls

        for index, row in calls.iterrows():
            strike = row['strike']
            market_price = row['lastPrice']
            iv = implied_volatility(market_price, current_price, strike, expiration, risk_free_rate)
            surface_data.append((expiration, strike, iv))

    # Store the surface data for each frame
    df_surface = pd.DataFrame(surface_data, columns=['Expiration', 'Strike', 'ImpliedVolatility']).dropna()

    x, y, z = df_surface['Strike'], df_surface['Expiration'], df_surface['ImpliedVolatility']
    x_grid = np.linspace(x.min(), x.max(), 100)
    y_grid = np.linspace(y.min(), y.max(), 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    Z = griddata((x, y), z, (X, Y), method='cubic')

    # Update norm for color scaling in each frame
    norm.vmin, norm.vmax = np.nanmin(Z), np.nanmax(Z)
    mappable.set_norm(norm)
    cbar.update_normal(mappable)

    # Plot the surface
    ax.clear()
    surf = ax.plot_surface(X, Y, Z/100, cmap='bone', edgecolor='none')
    ax.set_xlabel('Strike Price (USD)')
    ax.set_ylabel('Time to Expiry (years)')
    ax.set_zlabel('Implied Volatility')
    ax.set_title(f'Implied Volatility Surface for {ticker} at {interval_time} (Brent)')

    # Convert plot to image and append to images list for GIF
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(Image.fromarray(image))

# Save frames as GIF using Pillow
images[0].save(f"implied_volatility_surface_{ticker}_brent.gif", save_all=True, append_images=images[1:], duration=1000 // frames_per_second, loop=0)

plt.close()

<ipython-input-4-a4106445015e>:96: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
<ipython-input-4-a4106445015e>:96: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
<ipython-input-4-a4106445015e>:96: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
<ipython-input-4-a4106445015e>:96: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffe

In [ ]:
#Takes about 15 minutes to run
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime, timedelta
from scipy.interpolate import griddata
from PIL import Image

# Black-Scholes formula for option pricing
def black_scholes_call(S, K, T, r, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Vega function to compute the derivative of the option price with respect to volatility
def vega(S, K, T, r, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Function to calculate implied volatility using the Newton-Raphson method
def implied_volatility(option_price, S, K, T, r, option_type='call', max_iter=100, tol=1e-6):
    # Initial guess for volatility (usually in the range of 20%)
    sigma = 0.2

    for i in range(max_iter):
        if option_type == 'call':
            price = black_scholes_call(S, K, T, r, sigma)

        # Calculate the Vega (the derivative of the option price with respect to volatility)
        vega_value = vega(S, K, T, r, sigma)

        # Newton-Raphson update rule
        diff = price - option_price
        if abs(diff) < tol:
            return sigma  # Return the implied volatility if the difference is small enough

        sigma -= diff / vega_value  # Update sigma using the Newton-Raphson formula

    return np.nan  # Return NaN if the method fails to converge within the max iterations

# Parameters
ticker = "SPY"
risk_free_rate = 0.0425
frames_per_second = 4
interval = '15m'
history_duration = '5d'

# Download historical 15-minute interval data
stock = yf.Ticker(ticker)
historical_data = stock.history(interval=interval, period=history_duration)

# Get options data (using expiration dates)
options_dates = stock.options

# Prepare figure for each frame and save as image
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
cbar_ax = fig.add_axes([0.9, 0.15, 0.02, 0.7])  # Position for color bar
images = []  # List to store frames for GIF

# List to store surface data for animation
surface_data_frames = []
# Create a ScalarMappable for the color bar
norm = plt.Normalize(vmin=0, vmax=1)  # Initialize with dummy values
cmap = cm.bone
mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(mappable, cax=cbar_ax, label="Implied Volatility")

# Loop through each 15-minute interval to prepare data for each frame
for i in range(len(historical_data)):
    interval_time = historical_data.index[i]
    current_price = historical_data['Close'].iloc[i]

    surface_data = []
    for exp_date in options_dates[:5]:  # Limit to 5 expiration dates for simplicity
        expiration_date = pd.Timestamp(exp_date).tz_localize('America/New_York')
        expiration = (expiration_date - interval_time).days / 365.0  # Days to expiration in years

        options_chain = stock.option_chain(exp_date)
        calls = options_chain.calls

        for index, row in calls.iterrows():
            strike = row['strike']
            market_price = row['lastPrice']
            iv = implied_volatility(market_price, current_price, strike, expiration, risk_free_rate)
            surface_data.append((expiration, strike, iv))

    # Store the surface data for each frame
    df_surface = pd.DataFrame(surface_data, columns=['Expiration', 'Strike', 'ImpliedVolatility']).dropna()

    x, y, z = df_surface['Strike'], df_surface['Expiration'], df_surface['ImpliedVolatility']
    x_grid = np.linspace(x.min(), x.max(), 100)
    y_grid = np.linspace(y.min(), y.max(), 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    Z = griddata((x, y), z, (X, Y), method='cubic')

    # Plot the surface
    ax.clear()
    surf = ax.plot_surface(X, Y, Z/100, cmap='bone', edgecolor='none')
    ax.set_xlabel('Strike Price (USD)')
    ax.set_ylabel('Time to Expiry (years)')
    ax.set_zlabel('Implied Volatility')
    ax.set_title(f'Implied Volatility Surface for {ticker} at {interval_time} (Newton-Raphson)')

    # Convert plot to image and append to images list for GIF
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(Image.fromarray(image))

# Save frames as GIF using Pillow
images[0].save(f"implied_volatility_surface_{ticker}_newton.gif", save_all=True, append_images=images[1:], duration=1000 // frames_per_second, loop=0)

plt.close()

Streaming output truncated to the last 5000 lines.
<ipython-input-5-a0ca111e6548>:20: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
<ipython-input-5-a0ca111e6548>:40: RuntimeWarning: divide by zero encountered in scalar divide
  sigma -= diff / vega_value  # Update sigma using the Newton-Raphson formula
<ipython-input-5-a0ca111e6548>:13: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
<ipython-input-5-a0ca111e6548>:20: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
<ipython-input-5-a0ca111e6548>:40: RuntimeWarning: divide by zero encountered in scalar divide
  sigma -= diff / vega_value  # Update sigma using the Newton-Raphson formula
<ipython-input-5-a0ca111e6548>:13: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (